# Mini-ChatGPT

This tutorial assumes the reader has some basic machine learning (ML) knowledge, natural language processing (NLP) knowledge and familiarity with transformer neural network large language models, but is new to using reinforcement learning. If you know what a loss aka cost aka objective function is and how a token aka input_id represents a vector aka embedding, you should be in good shape.

If not, here is a really great place to start  https://lena-voita.github.io/nlp_course/language_modeling.html 

Special thanks to Leandro von Werra 's https://github.com/lvwerra/trl repo

diagram from https://openai.com/blog/chatgpt/ ChatGPT is an improved InstructGPT 

<img src='https://cdn.openai.com/chatgpt/draft-20221129c/ChatGPT_Diagram.svg' height=600 width=800>

#### Figure 1

### Step 1

Supervised Fine-Tuning (SFT) is the process of taking a model already pre-trained on a very large corpus of data, and re-training it carefully as not to erase the knowledge the model has gained from that pre-training [catastropic forgetting](https://github.com/clam004/intro_continual_learning), but rather slightly nudge the model to perform better in a more specific or narrow domain represented by a much smaller fine tuning dataset.

There are already plenty of resources on Supervised Fine-Tuning (SFT) aka fine-tuning already, some with impressively good explanations, mistakenly claiming to help you build a mini-chatgpt. I will not cover SFT, instead I will point you to some of my favorite resources. 

Here is an explanatory video by Huggingface of the Decoder, or autoregressive, transformers like GPT: https://youtu.be/d_ixlCubqQw

Here is a great blog explaining language modeling and the cross entropy loss function https://lena-voita.github.io/nlp_course/language_modeling.html , I would start here if you have done ML but not deep learning for NLP before. 

Fine-tuning GPT-like models: https://huggingface.co/course/chapter7/6?fw=pt and [Guide to fine-tuning Text Generation models: GPT-2, GPT-Neo and T5](https://towardsdatascience.com/guide-to-fine-tuning-text-generation-models-gpt-2-gpt-neo-and-t5-dc5de6b3bc5e)

ChatGPT likely was fine-tuned on a dataset of instructions and examples of following to
 
### Step 2

Fine-tuning usually means training for only a small number of epochs with a much smaller learning rate, but also using the cross entropy loss function used to do pre-training, but this way to learning is abit limited, can you tell why? 

<img src="https://lena-voita.github.io/resources/lectures/lang_models/neural/one_step_loss_intuition-min.png" height=500 width=700>

#### Figure 2

This example of using cross entropy shows how the training algorithm rewards or penalizes based on how large the logit (probability mass asigned to) for "cat" is, just this one way of completing the sequence, but we know that in reality when generating responses to instructions, there isnt one right ways to do it even in the cases when there is one right answer, there are many ways to be good and many ways to be bad. 

For the sake of keeping this tutorial light weight, fit on commonly available compute and intuitive, we have simplified the overall strategy in 2 major ways:

A. ChatGPT and InstructGPT, although they are very specially trained with reinforcement learning (RL), when you generalize, ChatGPT is just taking some prompt, aka input text (the phrase or instructions) and generating an output, aka continuing that text (providing a response or answer). It takes alot more compute memory to represent these long instructions and long answers in the neural network transformers, so instead we simplify the input text to the start of a movie review (first few words or subwords) and simplify the continuing text to the continuation of those first few words.

B. The reward modeling has been simplified as compared to InstructGPT https://openai.com/blog/instruction-following/ which is the model that ChatGPT is a scaled up and improved version of. 

Instead of training a reward model based on human rankings of output text, which was done to make the reward signal more stable or reliable, we use the more direct tactic of using the logits from a classifier (like BERT) as the reward signal, "positive meanss do more like this, negative means do less like this", where positive means the review is a referring to movie the reviewer thinks is good (a positive review). 

### Step 3

Given our limited time, I am going to focus on giving you what is hard to find. That is, good explainations of the parts that are usually glossed over yet important, or those parts that are usually explained in a much more jargony, field specific, technical or mathematical manner.

That means we will be focusing on step 3 in Figure 1. Namely, how reinforcement learning, something we have seen thus far in the popular media mostly applied to computer games, instead applied to natural language, or the generation of sequences of discrete tokens. 

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import torch

from matplotlib import pyplot as plt 

from minichatgpt.experiments.imdb import config, sent_kwargs
from minichatgpt import Lab, PPOConfig
from minichatgpt.processdata.collators import imdb_dataloader_collator

In [6]:
# The notebook should work even without GPUs, but if you have them, confirm you do
print ('number of GPUs:', torch.cuda.device_count())

# For the sake of the speed of this demonstration, the batch_size is temporarily decreased from 256 to 4
batch_size = 4
config.batch_size = batch_size
config.forward_batch_size = max(batch_size//8,1)

print('config.batch_size', config.batch_size)
print('config.forward_batch_size', config.forward_batch_size)

number of GPUs: 2
config.batch_size 4
config.forward_batch_size 1


In [7]:
lab = Lab(config)

dataset = lab.build_dataset(
    dataset_name="imdb",
    input_min_text_length=2,
    input_max_text_length=8,
)

new_policy, old_policy, tokenizer = lab.init_policies_tokenizer()

lab.set_generation_config(
    do_sample=True,
    output_min_length=4,
    output_max_length=16,
    pad_token_id=tokenizer.eos_token_id,
)

ppo_trainer = lab.init_ppo_trainer(
    config, 
    new_policy, 
    old_policy, 
    tokenizer, 
    dataset, 
    dataloader_collator = imdb_dataloader_collator,
)

reward_model = lab.init_reward_model()

Found cached dataset imdb (/home/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached processed dataset at /home/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-a680d0720b68191d.arrow
Loading cached processed dataset at /home/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-ec21da66149d9ccd.arrow


#### A new prompt is sampled from the dataset

In the cells below, samples of sentence text are represented by sequences integer token IDs in input_ids, and each token is either a word or a subword. Roughly speaking the average token represents a word or subword that is around 3 characters long: 'can', 'ed', 'con', 'tion' etc. Because of the code `input_min_text_length=2, input_max_text_length=8` above in `dataset = lab.build_dataset()`, you will find a random assortment of sample lengths in each batch, but they will be no less than 2 tokens short and no more than 8 tokens long. 

In reinforcement learning the agent acts in an environment, like a computer game, and takes actions, like hiting the jump button or moving to the left, which in the future may result in more or less reward, like points in a game. Here the environment is the random assortment of samples that the model can continue in many ways. The action here is how the model chooses to continue those samples, aka prompts.

In [8]:
for batch_step, batch in enumerate(ppo_trainer.dataloader):
    
    queries = batch['input_ids']
    
    break
    
print('the part of each batch are: ', batch.keys())
print('-'*50)
print('each batch has ', len(batch['query']), 'samples')
print('-'*50)
print('here are some samples examples ', batch['query'])
print('-'*50)
print('here are the token ids of those examples ', batch['input_ids'])

the part of each batch are:  dict_keys(['label', 'input_ids', 'query'])
--------------------------------------------------
each batch has  4 samples
--------------------------------------------------
here are some samples examples  ['I remember watching "G', 'I couldn', 'A suspenseful thriller that', 'I like musical']
--------------------------------------------------
here are the token ids of those examples  [tensor([  40, 3505, 4964,  366,   38], device='cuda:0'), tensor([  40, 3521], device='cuda:0'), tensor([   32, 43527,   913, 32251,   326], device='cuda:0'), tensor([   40,   588, 10530], device='cuda:0')]


#### The policy generates an output

run the next 2 cells several times, notice that the initial query (the prompt) stays the same, but the predicted actions (the text continuation) keeps changing, thats because in our generation keyword arguments we have `do_sample=True`

In [9]:
#### Get response from gpt2
responses = []
for query in queries:
    gen_len = lab.output_length_sampler()
    lab.generation_kwargs["max_new_tokens"] = gen_len
    response = ppo_trainer.generate(query, **lab.generation_kwargs)
    responses.append(response.squeeze()[-gen_len:])
    
responses

[tensor([27725,   286,   968,  1971,     1,   379,   257,  2318,    11],
        device='cuda:0'),
 tensor([ 470, 1650,  832,  428], device='cuda:0'),
 tensor([ 7622,   345,   422, 12598,   262, 15413,  1605], device='cuda:0'),
 tensor([ 6918,   828,   290,   314,  1549,   587,  2045,   329,  1194, 18304,
            13, 28796,  1106,  3941, 21429], device='cuda:0')]

In [10]:
batch['response'] = [tokenizer.decode(r.squeeze()) for r in responses]
concat_text = [batch['query'][i]+" >>> "+batch['response'][i] for i in range(len(batch['response']))]
concat_text

['I remember watching "G >>> angs of New York" at a bar,',
 "I couldn >>> 't sit through this",
 'A suspenseful thriller that >>>  keeps you from accepting the garbage American',
 "I like musical >>>  movies), and I'd been looking for another angel. wow.... Bill Pull"]

I used `>>>` to separate the prompt, left side, which is sampled from the dataset from the text generated by the policy, right side.

#### The reward model calculates a reward for the output

In [11]:
#### Compute sentiment score
texts = [q + r for q,r in zip(batch['query'], batch['response'])]
pipe_outputs = lab.reward_model(texts, **sent_kwargs)
rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

rewards

/home/carson/Desktop/Projects/minichatgpt/venv/lib/python3.7/site-packages/transformers/pipelines/text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


[tensor(0.9036), tensor(-2.4185), tensor(2.6272), tensor(0.2335)]

#### The reward is used to update the policy using PPO

PPO means proximal policy optimization. They are the reinforcement learning steps and they are all done by calling `stats = ppo_trainer.step(query_tensors, response_tensors, rewards)` in fact the entire Step 3 training loop is:

```python

for batch_step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    
    queries = batch['input_ids']
    
    #### Get response from gpt2
    responses = []
    for query in queries:
        gen_len = lab.output_length_sampler()
        lab.generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **lab.generation_kwargs)
        responses.append(response.squeeze()[-gen_len:])
        
    batch['response'] = [tokenizer.decode(r.squeeze()) for r in responses]
    
    #### Compute sentiment score
    texts = [q + r for q,r in zip(batch['query'], batch['response'])]
    pipe_outputs = lab.reward_model(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    
    #### Run PPO step 
    stats = ppo_trainer.step(queries, responses, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

```

But Im here to learn the details, so we will go through PPO as it applies to sequences of discrete tokens, step by step.

In [12]:
# check to make sure that each queries, responses and rewards is a list of tensors
queries, responses, scores = ppo_trainer._step_safety_checker(batch_size, queries, responses, rewards)

#### Policy model architecture 

To be clear on what happens next I find it helpful to remind myself of some of these details about our policy model. First, the policy is a causal language model with an added value head. We will talk about what the value head is later. 

```python

lab.init_policies_tokenizer()

    self.new_policy = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)

```

There are already so many great lessons on the internet about the architectures of transformers, here is a great place to start https://jalammar.github.io/illustrated-gpt2/ so I am not going to explain them here before making reference to them.  

The GPT like models are repeating blocks that pass their hidden state to the next block. The final or last hidden state (aka activations) is used as the input to the language model head (lm_head) and the value head (v_head).

simply run `new_policy` in the next cell to see for yourself

In [13]:
new_policy

AutoModelForCausalLMWithValueHead(
  (pretrained_model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
        

#### Preventing new policy from moving too far from old policy 

Next we find the log probabilities for each token in our responses, notice that the length of each element in the logprobs matches its corresponding pair in responses. We calculate the log probabilities not because we are implementing the reinforce policy gradient loss function [Policy Gradient Theorem](https://jonathan-hui.medium.com/rl-policy-gradients-explained-9b13b688b146), but in order to calculate the [Kullback–Leibler divergence](https://machinelearningmastery.com/divergence-between-probability-distributions/) (KL divergence) between the old static policy and new dynamic policy. You see, in 2022 the current efforts in RL are still being directed to combat the high variance and sample inefficiency in training, in laymens terms, trying to keep the neural network from spining out of control, losing its mind, while being updated, part of this involves keeping track of how much the behavior of the neural network in training has drifted away from the initial behavior, we measure this drift using the KL divergence between the two models outputs. [Great blog on the KL penalty](https://www.alignmentforum.org/posts/eoHbneGvqDu25Hasc/rl-with-kl-penalties-is-better-seen-as-bayesian-inference)

In [15]:
model_inputs = ppo_trainer.prepare_model_inputs(queries, responses)

model_inputs_names = list(model_inputs.keys())

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [16]:
with torch.no_grad():
    all_logprobs, _, values, masks = ppo_trainer.batched_forward_pass(
        ppo_trainer.model, 
        queries, 
        responses,
        model_inputs,
    )
    ref_logprobs, _, _, _ = ppo_trainer.batched_forward_pass(
        ppo_trainer.ref_model, 
        queries, 
        responses,
        model_inputs,
    )

In [17]:
values

tensor([[-0.0579,  2.8889,  1.1756,  1.0221, -0.0219,  0.6258,  0.6365, -0.8285,
          0.9817,  2.0409,  2.1924, -0.6101,  1.7262,  1.7949,  1.0600,  1.1138,
          0.9631],
        [-0.1871,  0.3736,  3.6847,  1.3886,  2.3806,  2.2053, -0.5814, -0.0348,
         -0.1332,  0.7509, -0.0424, -0.0572, -0.1253, -0.1446,  0.0921,  0.0761,
          0.0411],
        [-0.4117,  1.0689,  2.8009,  2.2523, -0.3216,  1.5736,  1.8668,  1.9978,
          2.2966,  1.9017,  1.8970,  1.9212,  0.8773, -0.0558,  0.0100, -0.0904,
          0.7808],
        [-0.1174,  2.3982,  1.5142,  2.4210,  2.5110,  0.5193,  2.3591,  3.6734,
          2.1235,  1.5990,  2.4188,  2.5484,  1.0261,  0.6882,  1.2205,  2.0228,
          0.1358]], device='cuda:0')

#### The output logits are policy's actions

If the hidden states size is 768, for example if `new_policy.pretrained_model.lm_head` prints `Linear(in_features=768, out_features=50257, bias=False)`, then that means we are using our last hidden state to predicet logits for each of the 50257 tokens in our vocabulary.

As you can seen below, the outputs of the policy are  `logits, loss, value = model(input_tokens)`

```python
class AutoModelForCausalLMWithValueHead(PreTrainedModelWrapper):
        .
        .
        .
    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        **kwargs,
    ):
        r"""
        Applies a forward pass to the wrapped model and returns the logits of the value head.
        Args:
            input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
                Indices of input sequence tokens in the vocabulary.
            past_key_values (`tuple(tuple(torch.FloatTensor))`, `optional`):
                Contains pre-computed hidden-states (key and values in the attention blocks) as computed by the model
                (see `past_key_values` input) to speed up sequential decoding.
            attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, `optional`):
                Mask to avoid performing attention on padding token indices. Mask values selected in ``[0, 1]``:
                - 1 for tokens that are **not masked**,
                - 0 for tokens that are **masked**.
            kwargs (`dict`, `optional`):
                Additional keyword arguments, that are passed to the wrapped model.
        """
        base_model_output = self.pretrained_model(
            input_ids=input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            output_hidden_states=True,  # We force the model to output hidden states
            **kwargs,
        )

        last_hidden_state = base_model_output.hidden_states[-1]
        lm_logits = base_model_output.logits
        loss = base_model_output.loss

        value = self.v_head(last_hidden_state).squeeze(-1)

        return (lm_logits, loss, value)
```

Logits are basically a non-normalized score for how likely a particular token is to come next. After the logits are pushed through a softmax layer they can sort of be interpreted as probabilities (horizontal bars below).

<img src="https://lena-voita.github.io/resources/lectures/lang_models/neural/nn_lm_idea_linear-min.png" height=600 width=800>

#### Figure 3.

The label "|V| tokens of them" in the diagram above is sayiing that there is a logit for every token in our vocabulary

The shape of the logits tensor is:

(batch_size, token_length_of_longest_sample_in_batch, number_of_tokens_in_vocabulary)

In [15]:
input_ids = ppo_trainer.data_collator(batch["input_ids"])["input_ids"]
input_kwargs = {"input_ids": input_ids}

input_ids

tensor([[   40,  3505,  4964,   366,    38],
        [   40,  3521, 50256, 50256, 50256],
        [   32, 43527,   913, 32251,   326],
        [   40,   588, 10530, 50256, 50256]], device='cuda:0')

In [16]:
logits, loss, values = new_policy(**input_kwargs)

print(logits[0,0,:]) # -15.1416
print(' ')
print(loss) 
print(' ')
print(values) # 0.0352,

logits.shape

tensor([-15.1416, -15.4052, -18.6318,  ..., -22.9367, -20.7308, -14.9200],
       device='cuda:0', grad_fn=<SliceBackward0>)
 
None
 
tensor([[-0.3129, -1.1940,  1.9523,  1.0476,  0.3108],
        [-0.0737, -2.4869,  0.5519,  1.6753,  1.0928],
        [-0.3067, -0.0982,  1.4994,  0.5941,  1.0739],
        [ 0.0921,  1.7487,  0.8781,  0.3169,  1.0723]], device='cuda:0',
       grad_fn=<SqueezeBackward1>)


torch.Size([4, 5, 50257])

#### Batched_forward_pass 

Below we dissect the operations done in `ppo_trainer.batched_forward_pass()` 

In the below cell I want to prepare you for some potentially confusing:

Step 3. 

```python
# Step 3. Get the log probabilities of the generated (sampled from logits) tokens 
logp = F.log_softmax(logits[:, :-1, :], dim=2)
logprobs = torch.gather(logp, 2, input_ids[:, 1:].unsqueeze(2)).squeeze(-1)
```

The blue diagram below, Blue Figure, is a tranformer and shows several blue boxes that represents the vector representations for each input token in the sequence horizontally and at each layer vertically. These are equivalent to the `h`'s in the grey green pink diagram above, Figure 3. 

It leaves out a a few steps between the deepest, aka last, or top most blue box and the word that comes out ot it. Implicit in Blue Figure is that just like in grey green pink Figure 3, in order for a token or word to be outputted from the top, the `h`'s go thru a linear layer to form `logits` and a softmax layer to form a vector of probabilities the same length as the number of possible tokens. Once you have this vector of probabilities, you have several options, you could choose the token with highest probability (ie a greedy approach) or sample based on those probabilities such that you have a higher chance of sampling reasonable tokens over unreasonable ones, ie `'I saw a cat on a'-> {'mat' > 'desk' > 'sofa' >>> 'cloud'} ` , this is what we chose, to sample, when we indicated `do_sample=True` above, which is why everytime you run the `response = ppo_trainer.generate(query, **lab.generation_kwargs)` you get different continuations of the beginning prompt text. 

The line of code `logp = F.log_softmax(logits[:, :-1, :], dim=2)` pushes the logits through both a softmax and then a natural log layer so that what you get is log probabilities `logp`. But why are we only concerned with the  `logp`'s before the last one `:-1` in `logits[:, :-1, :]` ? and why are we only concerned about the input IDs after the first one `1:` in `logprobs = torch.gather(logp, 2, input_ids[:, 1:].unsqueeze(2)).squeeze(-1)` ?

<img src="https://coriva.eu.org/images/nlp/lamdamodel.png" height=400 width=600>

#### Blue Figure.

Blue Figure supposes that tokens are words, notice how the word at the top of one column of boxes shows up at the bottom of the boxes of the next column just to the right of it. Basically they are saying that whatever token is sampled from the logits (softmaxed logits) gets incorporated into the newest, rightmost, part of the elongating sequence, and fed back into the transformer to generate the next set of logits to sample from and so on and so on. So if `t` is the position in the sequence, the token at position `t` is sampled from the logits at position `t - 1`, one position behind it. And since the torch.gather function is basically using the tokens in `input_ids`, to pick out the log probability in `logp` that corresponds to that token, we feed the `gather` function the `logits` and `input_ids` with `input_ids` shifted one position forward. 

here is a toy code snippet to show you have gather works, it assumes the batch size is 2, so 2 sample sequences, each sequence is 3 tokens long, and the vocabulary size is only 4. Instead of probabilities or log probabilties, we are just assuming we are indexing out the logits using our `ids` and that those logits are convienently `[1, 2, 3, 4]` for each position in the sequence.

```python
logits = torch.tensor([[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]],[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]]])
ids = torch.tensor([[0, 0, 1], [2, 1, 0]])
ids = ids.unsqueeze(2) # gather needs the dimension shape of logits and ids to be the same
print(ids.shape) # torch.Size([2, 3, 1])
selected_logits = torch.gather(logits, 2, ids)
print(selected_logits.shape) # torch.Size([2, 3, 1])
selected_logits = selected_logits.squeeze(-1) # we dont really need the last dimension, [2, 3, 1] -> [2, 3]
selected_logits 
```
result
```
tensor([[1, 1, 2],
        [3, 2, 1]])
```


In [17]:
import torch.nn.functional as F

all_logprobs = []
all_ref_logprobs = []
all_values = []

bs = lab.config.batch_size
fbs = lab.config.forward_batch_size

for i in range(int(bs / fbs)):
    
    query_batch = queries[i * fbs : (i + 1) * fbs]
    response_batch = responses[i * fbs : (i + 1) * fbs]
    
    sequence_tokens = [torch.cat([q, r]) for q, r in zip(query_batch, response_batch)]
    # Step 1. pads other sequences to the longest sample in the batch by 
    # adding tokenizer.pad_token (50256) to the right end of the sequence 
    input_ids = ppo_trainer.data_collator(sequence_tokens)["input_ids"]
    input_kwargs = {"input_ids": input_ids}
    
    # Step 2. Forward Pass through the model to get the logits
    with torch.no_grad():
        logits, _, v = new_policy(**input_kwargs)
        ref_logits, _, _ = old_policy(**input_kwargs)
    
    # Step 3. Get the log probabilities of the generated (sampled from logits) tokens 
    logp = F.log_softmax(logits[:, :-1, :], dim=2)
    logprobs = torch.gather(logp, 2, input_ids[:, 1:].unsqueeze(2)).squeeze(-1)

    logp = F.log_softmax(ref_logits[:, :-1, :], dim=2)
    ref_logprobs = torch.gather(logp, 2, input_ids[:, 1:].unsqueeze(2)).squeeze(-1)

    # Step 4. use the lenghts of the non-padded sequences to cut out the log probs
    # from the padded tokens and 
    for j in range(lab.config.forward_batch_size):

        start = len(query_batch[j]) - 1
        end = len(query_batch[j]) + len(response_batch[j]) - 1

        if len(logprobs[j, start:end]) < 2:
            raise ValueError("Responses are too short. Make sure they are at least 4 tokens long.")

        all_values.append(v[j, start - 1 : end - 1])
        all_logprobs.append(logprobs[j, start:end])
        all_ref_logprobs.append(ref_logprobs[j, start:end])
        
    break # stop after one forward_batch
    
all_logprobs

[tensor([-4.2014, -0.4262, -0.1335, -0.1198, -0.3778, -3.6512, -2.1602, -4.5329,
         -2.2796], device='cuda:0'),
 tensor([-2.1674e-03, -4.1700e+00, -2.8944e-01, -1.2249e+00], device='cuda:0')]

#### Compute rewards

$ R(x,y) = r(x,y) - \beta \log \frac{\pi(y|x)}{\rho(y|x)} $

We are using the [PPO2 algorithm](https://github.com/openai/baselines/tree/master/baselines/ppo2) with a modified reward, the log probabilities are used to calculate the KL divergence penalty on the reward with expectation $\beta KL(\pi, \rho)$, this penalty is described in equation (2) in [Fine-Tuning Language Models from Human Preferences](https://arxiv.org/pdf/1909.08593.pdf) where x is the prompt, y is the generated sequence, $ \pi $ is the distribution of the new policy, $ \rho $ is the distribution of the old policy, aka reference model, aka pretrained transformer, $KL(\pi, \rho)$ is the KL divergence between $ \rho $ and $ \pi $ measured in *nats* and $ \beta $ is dynamically adjusted to acheive a target $KL(\pi, \rho)$ such as *6 nats*.

In [18]:
# next we use these end of episode (end of sentence) rewards
logprobs, ref_logprobs, values = ppo_trainer.batched_forward_pass(queries, responses)
rewards, non_score_reward = ppo_trainer.compute_rewards(scores, logprobs, ref_logprobs)

rewards


[tensor([-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, 0.9036],
        device='cuda:0'),
 tensor([-0.0000, -0.0000, -0.0000, -2.4185], device='cuda:0'),
 tensor([-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, 2.6272],
        device='cuda:0'),
 tensor([-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, 0.2335], device='cuda:0')]

In [19]:
# inside the compute_rewards function
# before any updates to the new policy, it is the same as the old policy, so logprob_ - ref_logprob_ = 0 

rewards_, non_score_rewards_ = [], []

for score_, logprob_, ref_logprob_ in zip(scores, logprobs, ref_logprobs):
    kl = logprob_ - ref_logprob_ # kl = log[pi(y|x)/rho(y|x)]
    non_score_reward_ = -ppo_trainer.kl_ctl.value * kl # -beta * log[pi(y|x)/rho(y|x)]
    non_score_rewards_.append(non_score_reward_)
    reward_ = non_score_reward_.clone()
    reward_[-1] += score_ # the reward model score is added only to the last token in each sequence
    rewards_.append(reward_)
    print(reward_)

tensor([-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, 0.9036],
       device='cuda:0')
tensor([-0.0000, -0.0000, -0.0000, -2.4185], device='cuda:0')
tensor([-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, 2.6272],
       device='cuda:0')
tensor([-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
        -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, 0.2335], device='cuda:0')
